In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torchvision.datasets import Flowers102
import os
import numpy as np

In [2]:
# Load the Flowers102 dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Download the dataset (training split)
train_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='train',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Download the dataset (validation split)
val_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='val',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Download the dataset (test split)
test_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='test',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

In [3]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [4]:
# Define the MixUp function
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [5]:
# Create a MixUp data loader
def mixup_loader(dataloader, alpha=1.0):
    for data, target in dataloader:
        data, target_a, target_b, lam = mixup_data(data, target, alpha)
        yield data, target_a, target_b, lam

In [6]:
# Define the ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

# Modify the output layer to match the number of classes in the dataset (102 for Oxford Flowers)
model.fc = nn.Linear(model.fc.in_features, 102)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

no_improvement_counter = 0
patience = 5
for epoch in range(num_epochs):
    model.train()
    total = 0
    correct = 0
    running_loss = 0.0
    for data, target_a, target_b, lam in mixup_loader(train_loader):
        data, target_a, target_b = data.to(device), target_a.to(device), target_b.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = lam * criterion(outputs, target_a) + (1 - lam) * criterion(outputs, target_b)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += target_a.size(0)
        correct += (lam * predicted.eq(target_a.data).cpu().sum().float() + (1 - lam) * predicted.eq(target_b.data).cpu().sum().float())

    # Print training loss for each epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
    print(f"Training Accuracy: {100 * correct / total}%")

    # Validation
    model.eval()
    correct = 0
    total = 0
    validation_loss = 0
    best_validation_accuracy = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += predicted.eq(target.data).cpu().sum().float()

    # Print validation accuracy for each epoch
    val_accuracy = correct / total
    print(f"Validation Loss: {validation_loss / len(val_loader)}")
    print(f"Validation Accuracy: {100 * correct / total}%")

    # Early stopping
    if val_accuracy > best_validation_accuracy:
        best_validation_accuracy = val_accuracy
        no_improvement_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        no_improvement_counter += 1

    if no_improvement_counter >= patience:
        print("Early stopping: No improvement for {} epochs.".format(patience))
        break

# Save the trained model
torch.save(model.state_dict(), 'resnet50_flowers102_mixup.pth')

Epoch 1, Loss: 4.6821287125349045
Training Accuracy: 1.2290819883346558%
Validation Loss: 4.552988916635513
Validation Accuracy: 2.941176414489746%
Epoch 2, Loss: 4.554063558578491
Training Accuracy: 3.552126407623291%
Validation Loss: 4.415706738829613
Validation Accuracy: 10.29411792755127%
Epoch 3, Loss: 4.459620580077171
Training Accuracy: 7.412514686584473%
Validation Loss: 4.291141033172607
Validation Accuracy: 17.54901885986328%
Epoch 4, Loss: 4.337478503584862
Training Accuracy: 16.120710372924805%
Validation Loss: 4.151989296078682
Validation Accuracy: 28.52941131591797%
Epoch 5, Loss: 4.196594052016735
Training Accuracy: 27.983421325683594%
Validation Loss: 3.998853988945484
Validation Accuracy: 40.0%
Epoch 6, Loss: 4.079325683414936
Training Accuracy: 34.222660064697266%
Validation Loss: 3.8673679307103157
Validation Accuracy: 45.882354736328125%
Epoch 7, Loss: 4.030183382332325
Training Accuracy: 37.20534896850586%
Validation Loss: 3.766686365008354
Validation Accuracy: 49.

In [8]:
# Load the saved model (if not already loaded)
model.load_state_dict(torch.load('resnet50_flowers102_mixup.pth'))
model.to(device)
model.eval()  # Set the model to evaluation mode

test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss = 0.0
test_correct = 0
test_total = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        test_correct += predicted.eq(target.data).cpu().sum().float()

test_accuracy = 100 * test_correct / total
test_loss /= len(test_loader)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}%")


Test Loss: 1.1083086179328088
Test Accuracy: 83.07041931152344%


### More Data Augmentation

In [2]:
# Define data augmentation transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Randomly crop and resize
        transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
        transforms.RandomRotation(30),  # Randomly rotate by up to 30 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
        transforms.RandomGrayscale(p=0.2),  # Convert to grayscale with a 20% probability
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Random perspective transformation
        transforms.RandomVerticalFlip(),  # Randomly flip vertically
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  # Resize for validation and test
        transforms.CenterCrop(224),  # Center crop for validation and test
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),  # Resize for validation and test
        transforms.CenterCrop(224),  # Center crop for validation and test
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Apply the data augmentation transformations to your datasets
train_dataset = Flowers102(
    root='./data',
    split='train',
    transform=data_transforms['train'],  # Use data augmentation for training
    download=True
)

val_dataset = Flowers102(
    root='./data',
    split='val',
    transform=data_transforms['val'],  # Use validation data transformations
    download=True
)

test_dataset = Flowers102(
    root='./data',
    split='test',
    transform=data_transforms['test'],  # Use test data transformations
    download=True
)

# Create data loaders for training, validation, and test sets
batch_size = 64  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define the MixUp function
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Create a MixUp data loader
def mixup_loader(dataloader, alpha=1.0):
    for data, target in dataloader:
        data, target_a, target_b, lam = mixup_data(data, target, alpha)
        yield data, target_a, target_b, lam
        
# Define the ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

# Modify the output layer to match the number of classes in the dataset (102 for Oxford Flowers)
model.fc = nn.Linear(model.fc.in_features, 102)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

no_improvement_counter = 0
patience = 5
for epoch in range(num_epochs):
    model.train()
    total = 0
    correct = 0
    running_loss = 0.0
    for data, target_a, target_b, lam in mixup_loader(train_loader):
        data, target_a, target_b = data.to(device), target_a.to(device), target_b.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = lam * criterion(outputs, target_a) + (1 - lam) * criterion(outputs, target_b)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += target_a.size(0)
        correct += (lam * predicted.eq(target_a.data).cpu().sum().float() + (1 - lam) * predicted.eq(target_b.data).cpu().sum().float())

    # Print training loss for each epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
    print(f"Training Accuracy: {100 * correct / total}%")

    # Validation
    model.eval()
    correct = 0
    total = 0
    validation_loss = 0
    best_validation_accuracy = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += predicted.eq(target.data).cpu().sum().float()

    # Print validation accuracy for each epoch
    val_accuracy = correct / total
    print(f"Validation Loss: {validation_loss / len(val_loader)}")
    print(f"Validation Accuracy: {100 * correct / total}%")

    # Early stopping
    if val_accuracy > best_validation_accuracy:
        best_validation_accuracy = val_accuracy
        no_improvement_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        no_improvement_counter += 1

    if no_improvement_counter >= patience:
        print("Early stopping: No improvement for {} epochs.".format(patience))
        break

# Save the trained model
torch.save(model.state_dict(), 'resnet50_flowers102_mixup.pth')

Epoch 1, Loss: 4.682889610528946
Training Accuracy: 0.7725469470024109%
Validation Loss: 4.593235030770302
Validation Accuracy: 0.9803921580314636%
Epoch 2, Loss: 4.628222018480301
Training Accuracy: 1.229355812072754%
Validation Loss: 4.526539191603661
Validation Accuracy: 4.019608020782471%
Epoch 3, Loss: 4.589940443634987
Training Accuracy: 1.79043447971344%
Validation Loss: 4.459864974021912
Validation Accuracy: 6.56862735748291%
Epoch 4, Loss: 4.552561700344086
Training Accuracy: 3.189758539199829%
Validation Loss: 4.392927944660187
Validation Accuracy: 9.607843399047852%
Epoch 5, Loss: 4.509923219680786
Training Accuracy: 5.538421630859375%
Validation Loss: 4.326731137931347
Validation Accuracy: 13.13725471496582%
Epoch 6, Loss: 4.449300706386566
Training Accuracy: 9.377697944641113%
Validation Loss: 4.25895918160677
Validation Accuracy: 13.43137264251709%
Epoch 7, Loss: 4.4689967930316925
Training Accuracy: 6.746631622314453%
Validation Loss: 4.219809450209141
Validation Accurac

In [4]:
# Load the saved model (if not already loaded)
model.load_state_dict(torch.load('resnet50_flowers102_mixup.pth'))
model.to(device)
model.eval()  # Set the model to evaluation mode

test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss = 0.0
test_correct = 0
test_total = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        test_correct += predicted.eq(target.data).cpu().sum().float()

test_accuracy = 100 * test_correct / total
test_loss /= len(test_loader)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}%")

Test Loss: 1.9907307851808675
Test Accuracy: 62.270286560058594%


### Resnet-152

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torchvision.datasets import Flowers102
import os
import numpy as np

# Load the Flowers102 dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Download the dataset (training split)
train_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='train',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Download the dataset (validation split)
val_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='val',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Download the dataset (test split)
test_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='test',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define the MixUp function
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Create a MixUp data loader
def mixup_loader(dataloader, alpha=1.0):
    for data, target in dataloader:
        data, target_a, target_b, lam = mixup_data(data, target, alpha)
        yield data, target_a, target_b, lam

# Define the ResNet-152 model
model = models.resnet152(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

# Modify the output layer to match the number of classes in the dataset (102 for Oxford Flowers)
model.fc = nn.Linear(model.fc.in_features, 102)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

no_improvement_counter = 0
patience = 5
for epoch in range(num_epochs):
    model.train()
    total = 0
    correct = 0
    running_loss = 0.0
    for data, target_a, target_b, lam in mixup_loader(train_loader):
        data, target_a, target_b = data.to(device), target_a.to(device), target_b.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = lam * criterion(outputs, target_a) + (1 - lam) * criterion(outputs, target_b)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += target_a.size(0)
        correct += (lam * predicted.eq(target_a.data).cpu().sum().float() + (1 - lam) * predicted.eq(target_b.data).cpu().sum().float())

    # Print training loss for each epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
    print(f"Training Accuracy: {100 * correct / total}%")

    # Validation
    model.eval()
    correct = 0
    total = 0
    validation_loss = 0
    best_validation_accuracy = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += predicted.eq(target.data).cpu().sum().float()

    # Print validation accuracy for each epoch
    val_accuracy = correct / total
    print(f"Validation Loss: {validation_loss / len(val_loader)}")
    print(f"Validation Accuracy: {100 * correct / total}%")

    # Early stopping
    if val_accuracy > best_validation_accuracy:
        best_validation_accuracy = val_accuracy
        no_improvement_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        no_improvement_counter += 1

    if no_improvement_counter >= patience:
        print("Early stopping: No improvement for {} epochs.".format(patience))
        break

# Save the trained model
torch.save(model.state_dict(), 'resnet152_flowers102_mixup.pth')

Epoch 1, Loss: 4.686394557356834
Training Accuracy: 0.7857245802879333%
Validation Loss: 4.578508660197258
Validation Accuracy: 1.372549057006836%
Epoch 2, Loss: 4.547796651721001
Training Accuracy: 3.1319284439086914%
Validation Loss: 4.410183101892471
Validation Accuracy: 9.70588207244873%
Epoch 3, Loss: 4.444122940301895
Training Accuracy: 7.908377170562744%
Validation Loss: 4.274492472410202
Validation Accuracy: 22.254901885986328%
Epoch 4, Loss: 4.288861200213432
Training Accuracy: 18.87061309814453%
Validation Loss: 4.12100300937891
Validation Accuracy: 32.156864166259766%
Epoch 5, Loss: 4.155911609530449
Training Accuracy: 28.634626388549805%
Validation Loss: 3.958118736743927
Validation Accuracy: 42.156864166259766%
Epoch 6, Loss: 4.0417996644973755
Training Accuracy: 33.79977035522461%
Validation Loss: 3.8295202925801277
Validation Accuracy: 44.80392074584961%
Epoch 7, Loss: 3.9592252746224403
Training Accuracy: 38.86756896972656%
Validation Loss: 3.7053838670253754
Validation

In [3]:
# Load the saved model (if not already loaded)
model.load_state_dict(torch.load('resnet152_flowers102_mixup.pth'))
model.to(device)
model.eval()  # Set the model to evaluation mode

test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss = 0.0
test_correct = 0
test_total = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        test_correct += predicted.eq(target.data).cpu().sum().float()

test_accuracy = 100 * test_correct / total
test_loss /= len(test_loader)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}%")


Test Loss: 1.2569124533413605
Test Accuracy: 79.29744720458984%
